In [6]:
import spotlight
import pandas as pd
import urllib, json
from collections import Counter

In [8]:
dfsm = pd.read_csv('nhsconjoin.csv')
dfsm

,Unnamed: 0,1,2
0,1,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),..."
1,2,Acne,periods – some women have a flare-up of acne j...
2,3,Acute cholecystitis,"a high temperature (fever),nausea and vomiting..."
3,4,Acute lymphoblastic leukaemia,"pale skin,feeling tired and breathless,having ..."
4,5,Acute lymphoblastic leukaemia: Children,"red blood cells, which carry oxygen around the..."
5,6,Acute lymphoblastic leukaemia: Teenagers and y...,white blood cells – which help us fight infect...
6,7,Acute myeloid leukaemia,"pale skin,tiredness,breathlessness,a high temp..."
7,8,Acute myeloid leukaemia: Children,"red blood cells, which carry oxygen around the..."
8,9,Acute myeloid leukaemia: Teenagers and young a...,white blood cells – which help us fight infect...
9,10,Acute pancreatitis,"nausea (feeling sick) or vomiting,diarrhoea,in..."


# Invoke spotlight

In [100]:
annotations = spotlight.annotate(restAPI,text1, confidence=0.6, support=20)

In [1]:
#for anni in annotations:
    #print(anni['URI'])

a pulsating feeling in your stomach (abdomen), usually near your belly button, that's usually only noticeable when you touch it,persistent back pain,persistent abdominal pain,dizziness,sweaty and clammy skin,rapid heartbeat (tachycardia),shortness of breath,feeling faint,loss of consciousness

In [5]:
import unicodedata
dfsm['1'][1]

'Acne'

In [67]:
aDict = {}
for ic in range(5,6):
    conditionname = dfsm['1'][ic].split(':')
    dbpediaText = ''
    if (len(conditionname)>1):  
        conditionname = conditionname[0].replace(' ','_')
    else:
        if (len(conditionname)>3):
            conditionTerm = conditionname[0]+'_'+conditionname[1]+'_'+conditionname[2]
        else: 
            conditionTerm = dfsm['1'][ic].replace(' ','_')
    dbpdialink = 'http://dbpedia.org/data/'+conditionTerm+'.json'
    print(dbpdialink)
    icresponse = urllib.urlopen(dbpdialink)
    jicdata = json.loads(icresponse.read())
    firstkey=jicdata['http://dbpedia.org/resource/'+conditionTerm]
    if 'http://dbpedia.org/ontology/abstract' in firstkey:
        secondkey = firstkey['http://dbpedia.org/ontology/abstract']
        for abstract in secondkey:
            if abstract['lang'] == 'en':
                dbpediaText=abstract['value']
    else:
        if 'http://dbpedia.org/ontology/wikiPageRedirects' in firstkey:
            secondkey = firstkey['http://dbpedia.org/ontology/wikiPageRedirects']
            dbpdiaredir = secondkey[0]['value'].replace('resource','data')+".json"
            print(dbpdiaredir)
            icresponse = urllib.urlopen(dbpdiaredir)
            jicdata = json.loads(icresponse.read())
            firstkey=jicdata[secondkey[0]['value']]
            if 'http://dbpedia.org/ontology/abstract' in firstkey:
                secondkey = firstkey['http://dbpedia.org/ontology/abstract']
                for abstract in secondkey:
                    if abstract['lang'] == 'en':
                        dbpediaText=abstract['value']
    if (dbpediaText!=''):
        annotations = spotlight.annotate(restAPI,dbpediaText, confidence=0.4, support=20)
        jointerms = []
        for terms in annotations: 
            uniterms = unicodedata.normalize('NFKD', terms['URI']).encode('ascii','ignore')
            jointerms.append(uniterms)
            aDict[dfsm['1'][ic]] = jointerms
            
    else:
        print('no term find')
       

http://dbpedia.org/data/Acute_lymphoblastic_leukaemia.json
http://dbpedia.org/data/Acute_lymphoblastic_leukemia.json


In [68]:
semanticdf = pd.DataFrame(list(aDict.items()),columns=['1','dbp'])
semanticdf

,1,dbp
0,Acute lymphoblastic leukaemia: Teenagers and y...,[http://dbpedia.org/resource/Acute_lymphoblast...


In [73]:
TaDict = {}
for i in range(5,6):
    texts=dfsm['2'][i]
    annotations = spotlight.annotate(restAPI,texts, confidence=0.2, support=20)
    Tjointerms = []
    for terms in annotations: 
        Tniterms = unicodedata.normalize('NFKD', terms['URI']).encode('ascii','ignore')
        Tjointerms.append(Tniterms)
    Tjointerms=list(dict.fromkeys(Tjointerms))
    TaDict[dfsm['1'][i]] = Tjointerms

In [74]:
Tsemanticdf = pd.DataFrame(list(TaDict.items()),columns=['1','dpb annoation'])


In [75]:
Tsemanticdf

,1,dpb annoation
0,Acute lymphoblastic leukaemia: Teenagers and y...,"[http://dbpedia.org/resource/Infection, http:/..."


In [76]:
semanticdf['dbp'][0][0]

'http://dbpedia.org/resource/Acute_lymphoblastic_leukemia'

In [77]:
sdf = pd.merge(semanticdf, Tsemanticdf, on='1')
sdf

,1,dbp,dpb annoation
0,Acute lymphoblastic leukaemia: Teenagers and y...,[http://dbpedia.org/resource/Acute_lymphoblast...,"[http://dbpedia.org/resource/Infection, http:/..."


In [78]:
sl = Counter()
for idx in sdf.index:
    sl.update(sdf.loc[idx, "dpb annoation"])  
sl.most_common(10)

[('http://dbpedia.org/resource/Subroutine', 1),
 ('http://dbpedia.org/resource/Genetic_carrier', 1),
 ('http://dbpedia.org/resource/Bleeding', 1),
 ('http://dbpedia.org/resource/Leukemia', 1),
 ('http://dbpedia.org/resource/Myeloblast', 1),
 ('http://dbpedia.org/resource/Thrombus', 1),
 ('http://dbpedia.org/resource/Human_body', 1),
 ('http://dbpedia.org/resource/Lymphoblast', 1),
 ('http://dbpedia.org/resource/Red_blood_cell', 1),
 ('http://dbpedia.org/resource/Myelocyte', 1)]

In [84]:
testdf = pd.read_csv('semanndf1.csv')
testdf

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,1,2,dbp,dpb annoation
0,0,0,1,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),...",[http://dbpedia.org/resource/Abdominal_aortic_...,"[http://dbpedia.org/resource/Variable_star, ht..."
1,1,1,2,Acne,periods – some women have a flare-up of acne j...,"[http://dbpedia.org/resource/Acne_vulgaris, ht...",[http://dbpedia.org/resource/Glossary_of_profe...


In [85]:
testdf = testdf.drop(testdf.columns[[0, 1]], axis=1)
testdf

,Unnamed: 0.1.1,1,2,dbp,dpb annoation
0,1,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),...",[http://dbpedia.org/resource/Abdominal_aortic_...,"[http://dbpedia.org/resource/Variable_star, ht..."
1,2,Acne,periods – some women have a flare-up of acne j...,"[http://dbpedia.org/resource/Acne_vulgaris, ht...",[http://dbpedia.org/resource/Glossary_of_profe...


In [81]:
testdf.to_csv("semanndf1.csv", encoding="utf-8")

In [82]:
finaldf = pd.read_csv('semanndf1.csv')
finaldf = finaldf.drop(finaldf.columns[[0]], axis=1)

In [83]:
finaldf

,Unnamed: 0.1,Unnamed: 0.1.1,1,2,dbp,dpb annoation
0,0,1,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),...",[http://dbpedia.org/resource/Abdominal_aortic_...,"[http://dbpedia.org/resource/Variable_star, ht..."
1,1,2,Acne,periods – some women have a flare-up of acne j...,"[http://dbpedia.org/resource/Acne_vulgaris, ht...",[http://dbpedia.org/resource/Glossary_of_profe...


In [61]:
dfsm1 = dfsm.drop(dfsm.columns[[0]], axis=1)
dfsm1.head(5)

,1,2
0,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),..."
1,Acne,periods – some women have a flare-up of acne j...
2,Acute cholecystitis,"a high temperature (fever),nausea and vomiting..."
3,Acute lymphoblastic leukaemia,"pale skin,feeling tired and breathless,having ..."
4,Acute lymphoblastic leukaemia: Children,"red blood cells, which carry oxygen around the..."


In [62]:
sdf

,1,dbp,dpb annoation
0,Acute cholecystitis,"[http://dbpedia.org/resource/Cholecystitis, ht...","[http://dbpedia.org/resource/Peritonitis, http..."
1,Acute lymphoblastic leukaemia: Children,[http://dbpedia.org/resource/Acute_lymphoblast...,[http://dbpedia.org/resource/B-cell_chronic_ly...
2,Acute lymphoblastic leukaemia,[http://dbpedia.org/resource/Acute_lymphoblast...,"[http://dbpedia.org/resource/Short_film, http:..."


In [65]:
finaldf = finaldf.append(pd.merge(dfsm1, sdf, on='1')).reset_index()

In [66]:
finaldf

,index,1,2,dbp,dpb annoation
0,0,Abdominal aortic aneurysm,"a pulsating feeling in your stomach (abdomen),...",[http://dbpedia.org/resource/Abdominal_aortic_...,"[http://dbpedia.org/resource/Variable_star, ht..."
1,1,Acne,periods – some women have a flare-up of acne j...,"[http://dbpedia.org/resource/Acne_vulgaris, ht...",[http://dbpedia.org/resource/Glossary_of_profe...
2,0,Acute cholecystitis,"a high temperature (fever),nausea and vomiting...","[http://dbpedia.org/resource/Cholecystitis, ht...","[http://dbpedia.org/resource/Peritonitis, http..."
3,1,Acute lymphoblastic leukaemia,"pale skin,feeling tired and breathless,having ...",[http://dbpedia.org/resource/Acute_lymphoblast...,"[http://dbpedia.org/resource/Short_film, http:..."
4,2,Acute lymphoblastic leukaemia: Children,"red blood cells, which carry oxygen around the...",[http://dbpedia.org/resource/Acute_lymphoblast...,[http://dbpedia.org/resource/B-cell_chronic_ly...
5,0,Acute cholecystitis,"a high temperature (fever),nausea and vomiting...","[http://dbpedia.org/resource/Cholecystitis, ht...","[http://dbpedia.org/resource/Peritonitis, http..."
6,1,Acute lymphoblastic leukaemia,"pale skin,feeling tired and breathless,having ...",[http://dbpedia.org/resource/Acute_lymphoblast...,"[http://dbpedia.org/resource/Short_film, http:..."
7,2,Acute lymphoblastic leukaemia: Children,"red blood cells, which carry oxygen around the...",[http://dbpedia.org/resource/Acute_lymphoblast...,[http://dbpedia.org/resource/B-cell_chronic_ly...


In [80]:
finaldf.to_csv("semanndf.csv", encoding="utf-8")

In [23]:
print(type(sdf['dpb annoation']))

<class 'pandas.core.series.Series'>


In [108]:
urlstr = semanticdf['dbp'][0][2].replace("resource","data")+".json"
print (urlstr)

http://dbpedia.org/data/Nausea.json


In [109]:
response = urllib.urlopen(urlstr)
jdata = json.loads(response.read())
firstkey=jdata[semanticdf['dbp'][0][2]]
print firstkey

{u'http://dbpedia.org/property/field': [{u'type': u'uri', u'value': u'http://dbpedia.org/resource/Gastroenterology'}], u'http://dbpedia.org/property/meshid': [{u'datatype': u'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString', u'type': u'literal', u'value': u'D009325'}], u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [{u'type': u'uri', u'value': u'http://dbpedia.org/ontology/Disease'}], u'http://purl.org/dc/terms/subject': [{u'type': u'uri', u'value': u'http://dbpedia.org/resource/Category:Vomiting'}, {u'type': u'uri', u'value': u'http://dbpedia.org/resource/Category:Symptoms_and_signs:_Digestive_system_and_abdomen'}], u'http://purl.org/linguistics/gold/hypernym': [{u'type': u'uri', u'value': u'http://dbpedia.org/resource/Sensation'}], u'http://www.w3.org/2000/01/rdf-schema#comment': [{u'lang': u'de', u'type': u'literal', u'value': u'Nausea (deutsch \xdcbelkeit, lat. nausea, sp\xe4tgriechisch \u1f21 \u03bd\u03b1\u1fe6\u03c4\u03af\u03b1 naut\xeda \u201eSeekrankheit\u201c, zu al

In [110]:
secondkey = firstkey['http://dbpedia.org/ontology/abstract']
for abstract in secondkey:
    if abstract['lang'] == 'en':
        print(abstract['value'])

Nausea is a sensation of unease and discomfort in the upper stomach with an involuntary urge to vomit. It may precede vomiting, but a person can have nausea without vomiting. When prolonged, it is a debilitating symptom. Nausea is a non-specific symptom, which means that it has many possible causes. Some common causes of nausea are motion sickness, dizziness, migraine, fainting, gastroenteritis (stomach infection) or food poisoning. Nausea is a side effect of many medications including chemotherapy, nauseants or morning sickness in early pregnancy. Nausea may also be caused by anxiety, disgust and depression. Medications taken to prevent and treat nausea are called antiemetics. The most commonly prescribed antiemetics in the US are promethazine, metoclopramide and ondansetron. The word nausea is from Latin nausea, from Greek ναυσία – nausia, "ναυτία" – nautia, motion sickness", "feeling sick or queasy".


In [2]:
#for termRdf in annRdf:
   #print(termRdf['URI'])
    